## TensorFlow中的卷积

在TensorFlow中去做卷积，我们有很多内建的层可以使用。你可以输入2维数据做1维卷积，输入3维数据做2维卷积，输入4维数据做3维卷积，最常用的是2维卷积。

```python
# 函数模型
tf.nn.conv2d(
    input,
    filter,
    strides,
    padding,
    use_cudnn_on_gpu=True,
    data_format='NHWC',
    dilations=[1, 1, 1, 1],
    name=None)

Input: Batch size (N) x Height (H) x Width (W) x Channels (C)
Filter: Height x Width x Input Channels x Output Channels
(e.g. [5, 5, 3, 64])
Strides: 4 element 1-D tensor, strides in each direction
(often [1, 1, 1, 1] or [1, 2, 2, 1])
Padding: 'SAME' or 'VALID'
Dilations: The dilation factor. If set to k > 1, there will be k-1 skipped cells between each filter element on that dimension.
Data_format: default to NHWC
```

作一个有趣的练习：GitHub中的kernes.py文件中看到一些著名的核的值，在07_run_kernels.py中看到它们的用法。

## 用CNN处理MNIST

在第三课中学习了逻辑回归处理MNIST，现在我们使用CNN来处理，看看结果如何！

将采用如下架构：两个步长为1的卷积层，每个卷积层后跟一个relu激活层与最大池化层Maxpool，最后跟两个全连接层。

### 1.卷积层


- 输入尺寸(W)
- 过滤器尺寸(F)
- 步长(S)
- 零填充(P)

在定义函数之前，让我们看一下获取输出大小的公式。当您具有上述输入值时，输出的大小如下所示：

$$ \frac{W-F+2P}{S}+1 $$


在我们的MNIST模型中，输入为28x28，滤波器为5x5。并且步幅使用1和填充使用2。因此，输出的大小如下:

$$ \frac{28-5+2\times2}{1}+1 = 28 $$

In [1]:
def conv_relu(inputs, filters, k_size, stride, padding, scope_name):
    with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        # rgb通道
        in_channels = inputs.shape[-1]
        # 卷积核
        kernel = tf.get_variable('kernel', [k_size, k_size, in_channels, filters],
                                initializer=tf.truncated_normal_initializer())
        biases = tf.get_variable('biases', [filters],
                            initializer=tf.random_normal_initializer())
        # 卷积结果
        conv = tf.nn.conv2d(inputs, kernel, strides=[1, stride, stride, 1], padding=padding)
    # relu层对卷积结果处理
    return tf.nn.relu(conv + biases, name=scope.name)

### 2.池化层

池化可减少要素图的维数，提取要素并缩短执行时间。

通常使用max-pooling或average-pooling。

由于在此模型中使用了max-pooling，因此我们定义了max-pooling函数，如下所示:


- 输入尺寸（W）
- 池化大小（K）
- 池化步长（S）
- 池化零填充（P）


$$ \frac{W-K+2P}{S}+1 $$


在我们的模型中，输入是28x28，池大小是2x2，补长是2，零填充，所以我们将输出大小如下。

$$ \frac{28-2+2\times0}{2}+1=14 $$

In [2]:
def maxpool(inputs, ksize, stride, padding='VALID', scope_name='pool'):
    with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        pool = tf.nn.max_pool(inputs,
                            ksize=[1, ksize, ksize, 1],
                            strides=[1, stride, stride, 1],
                            padding=padding)
    return pool

### 3.全连接层

In [3]:
def fully_connected(inputs, out_dim, scope_name='fc'):
    with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        in_dim = inputs.shape[-1]
        w = tf.get_variable('weights', [in_dim, out_dim],
                            initializer=tf.truncated_normal_initializer())
        b = tf.get_variable('biases', [out_dim],
                            initializer=tf.constant_initializer(0.0))
        out = tf.matmul(inputs, w) + b
    return out

### 4.组合调用

现在让我们通过组合我们创建的函数来创建整个模型。您可以使用我们按顺序创建的功能。

需要注意的一点是，当您在最后一次池化后转到fc层时，必须通过将一维向量的大小乘以原始数组的每个维度的长度来重新整形三维数组的一维数组。

最后，将dropout应用到fc层。

In [4]:
def inference(self):
        conv1 = conv_relu(inputs=self.img,
                        filters=32,
                        k_size=5,
                        stride=1,
                        padding='SAME',
                        scope_name='conv1')
        pool1 = maxpool(conv1, 2, 2, 'VALID', 'pool1')
        conv2 = conv_relu(inputs=pool1,
                        filters=64,
                        k_size=5,
                        stride=1,
                        padding='SAME',
                        scope_name='conv2')
        pool2 = maxpool(conv2, 2, 2, 'VALID', 'pool2')
        feature_dim = pool2.shape[1] * pool2.shape[2] * pool2.shape[3]
        pool2 = tf.reshape(pool2, [-1, feature_dim])
        fc = tf.nn.relu(fully_connected(pool2, 1024, 'fc'))
        dropout = tf.layers.dropout(fc, self.keep_prob, training=self.training, name='dropout')

        self.logits = fully_connected(dropout, self.n_classes, 'logits')

### 5.loss

In [ ]:
def loss(self):
        '''
        define loss function
        use softmax cross entropy with logits as the loss function
        compute mean cross entropy, softmax is applied internally
        '''
        # 
        with tf.name_scope('loss'):
            entropy = tf.nn.softmax_cross_entropy_with_logits(labels=self.label, logits=self.logits)
            self.loss = tf.reduce_mean(entropy, name='loss')

### 6.评估

在训练时，需要评估每个epoch的准确率。

In [14]:
def eval(self):
        '''
        Count the number of right predictions in a batch
        '''
        with tf.name_scope('predict'):
            preds = tf.nn.softmax(self.logits)
            correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(self.label, 1))
            self.accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))


### 7.封装

In [4]:
import tensorflow as tf
import utils
import os
import time
class ConvNet(object):
    def __init__(self):
        self.lr = 0.001
        self.batch_size = 128
        self.keep_prob = tf.constant(0.75)
        self.gstep = tf.Variable(0, dtype=tf.int32, 
                                trainable=False, name='global_step')
        self.n_classes = 10
        self.skip_step = 20
        self.n_test = 10000
        self.training = True

    def get_data(self):
        with tf.name_scope('data'):
            train_data, test_data = utils.get_mnist_dataset(self.batch_size)
            iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                                   train_data.output_shapes)
            img, self.label = iterator.get_next()
            self.img = tf.reshape(img, shape=[-1, 28, 28, 1])
            # reshape the image to make it work with tf.nn.conv2d

            self.train_init = iterator.make_initializer(train_data)  # initializer for train_data
            self.test_init = iterator.make_initializer(test_data)    # initializer for train_data

    def inference(self):
        conv1 = conv_relu(inputs=self.img,
                        filters=32,
                        k_size=5,
                        stride=1,
                        padding='SAME',
                        scope_name='conv1')
        print("conv:"+str(conv1))
        pool1 = maxpool(conv1, 2, 2, 'VALID', 'pool1')
        print("pool1:"+str(pool1))
        conv2 = conv_relu(inputs=pool1,
                        filters=64,
                        k_size=5,
                        stride=1,
                        padding='SAME',
                        scope_name='conv2')
        print("conv2:"+str(conv2))
        pool2 = maxpool(conv2, 2, 2, 'VALID', 'pool2')
        print("pool2:"+str(pool2))
        feature_dim = pool2.shape[1] * pool2.shape[2] * pool2.shape[3]
        print("feature_dim:"+str(feature_dim))
        pool2 = tf.reshape(pool2, [-1, feature_dim])
        print("pool2:"+str(pool2))
        fc = fully_connected(pool2, 1024, 'fc')
        print("fc:"+str(fc))
        dropout = tf.nn.dropout(tf.nn.relu(fc), self.keep_prob, name='relu_dropout')
        print("dropout:"+str(dropout))
        self.logits = fully_connected(dropout, self.n_classes, 'logits')
        print("self.logits:"+str(self.logits))

    def loss(self):
        '''
        define loss function
        use softmax cross entropy with logits as the loss function
        compute mean cross entropy, softmax is applied internally
        '''
        # 
        with tf.name_scope('loss'):
            entropy = tf.nn.softmax_cross_entropy_with_logits(labels=self.label, logits=self.logits)
            self.loss = tf.reduce_mean(entropy, name='loss')
    
    def optimize(self):
        '''
        Define training op
        using Adam Gradient Descent to minimize cost
        '''
        self.opt = tf.train.AdamOptimizer(self.lr).minimize(self.loss, 
                                                global_step=self.gstep)

    def summary(self):
        '''
        Create summaries to write on TensorBoard
        '''
        with tf.name_scope('summaries'):
            tf.summary.scalar('loss', self.loss)
            tf.summary.scalar('accuracy', self.accuracy)
            tf.summary.histogram('histogram loss', self.loss)
            self.summary_op = tf.summary.merge_all()
    
    def eval(self):
        '''
        Count the number of right predictions in a batch
        '''
        with tf.name_scope('predict'):
            preds = tf.nn.softmax(self.logits)
            correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(self.label, 1))
            self.accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

    def build(self):
        '''
        Build the computation graph
        '''
        self.get_data()
        self.inference()
        self.loss()
        self.optimize()
        self.eval()
        self.summary()

    def train_one_epoch(self, sess, saver, init, writer, epoch, step):
        start_time = time.time()
        sess.run(init) 
        self.training = True
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l, summaries = sess.run([self.opt, self.loss, self.summary_op])
                writer.add_summary(summaries, global_step=step)
                if (step + 1) % self.skip_step == 0:
                    print('Loss at step {0}: {1}'.format(step, l))
                step += 1
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        saver.save(sess, 'checkpoints/convnet_mnist/mnist-convnet', step)
        print('Average loss at epoch {0}: {1}'.format(epoch, total_loss/n_batches))
        print('Took: {0} seconds'.format(time.time() - start_time))
        return step

    def eval_once(self, sess, init, writer, epoch, step):
        start_time = time.time()
        sess.run(init)
        self.training = False
        total_correct_preds = 0
        try:
            while True:
                accuracy_batch, summaries = sess.run([self.accuracy, self.summary_op])
                writer.add_summary(summaries, global_step=step)
                total_correct_preds += accuracy_batch
        except tf.errors.OutOfRangeError:
            pass

        print('Accuracy at epoch {0}: {1} '.format(epoch, total_correct_preds/self.n_test))
        print('Took: {0} seconds'.format(time.time() - start_time))

    def train(self, n_epochs):
        '''
        The train function alternates between training one epoch and evaluating
        '''
        utils.safe_mkdir('checkpoints')
        utils.safe_mkdir('checkpoints/convnet_mnist')
        writer = tf.summary.FileWriter('./graphs/convnet', tf.get_default_graph())

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            saver = tf.train.Saver()
            ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/convnet_mnist/checkpoint'))
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
            
            step = self.gstep.eval()

            for epoch in range(n_epochs):
                step = self.train_one_epoch(sess, saver, self.train_init, writer, epoch, step)
                self.eval_once(sess, self.test_init, writer, epoch, step)
        writer.close()

if __name__ == '__main__':
    model = ConvNet()
    model.build()
    model.train(n_epochs=30)

data/mnist/train-images-idx3-ubyte.gz already exists
data/mnist/train-labels-idx1-ubyte.gz already exists
data/mnist/t10k-images-idx3-ubyte.gz already exists
data/mnist/t10k-labels-idx1-ubyte.gz already exists
conv:Tensor("conv1_1:0", shape=(?, 28, 28, 32), dtype=float32)
pool1:Tensor("pool1/MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
conv2:Tensor("conv2_1:0", shape=(?, 14, 14, 64), dtype=float32)
pool2:Tensor("pool2/MaxPool:0", shape=(?, 7, 7, 64), dtype=float32)
feature_dim:3136
pool2:Tensor("Reshape:0", shape=(?, 3136), dtype=float32)
fc:Tensor("fc/add:0", shape=(?, 1024), dtype=float32)
dropout:Tensor("relu_dropout/mul:0", shape=(?, 1024), dtype=float32)
self.logits:Tensor("logits/add:0", shape=(?, 10), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

INFO:tensorflow:Summary name histogram loss is illegal; using histogra

Loss at step 3339: 32.812469482421875
Loss at step 3359: 9.880901336669922
Loss at step 3379: 37.180015563964844
Loss at step 3399: 20.444835662841797
Loss at step 3419: 25.90365219116211
Loss at step 3439: 11.969549179077148
Average loss at epoch 7: 42.81812903160272
Took: 15.069268465042114 seconds
Accuracy at epoch 7: 0.9686 
Took: 0.8306162357330322 seconds
Loss at step 3459: 102.39554595947266
Loss at step 3479: 46.702369689941406
Loss at step 3499: 0.0
Loss at step 3519: 27.893085479736328
Loss at step 3539: 4.3278045654296875
Loss at step 3559: 4.8789825439453125
Loss at step 3579: 44.614688873291016
Loss at step 3599: 0.0
Loss at step 3619: 95.66297912597656
Loss at step 3639: 41.93891906738281
Loss at step 3659: 81.085205078125
Loss at step 3679: 38.55162048339844
Loss at step 3699: 28.404909133911133
Loss at step 3719: 40.942832946777344
Loss at step 3739: 48.79941177368164
Loss at step 3759: 44.761146545410156
Loss at step 3779: 21.152957916259766
Loss at step 3799: 24.45763

Loss at step 7319: 2.5546417236328125
Loss at step 7339: 12.190032958984375
Loss at step 7359: 24.120805740356445
Loss at step 7379: 7.202362060546875
Loss at step 7399: 0.0
Loss at step 7419: 0.0
Loss at step 7439: 0.0
Loss at step 7459: 0.0
Loss at step 7479: 17.14218521118164
Loss at step 7499: 0.0
Loss at step 7519: 0.0
Loss at step 7539: 4.968559265136719
Loss at step 7559: 2.9908065795898438
Loss at step 7579: 1.8168716430664062
Loss at step 7599: 3.7762832641601562
Loss at step 7619: 31.4625301361084
Loss at step 7639: 0.0
Loss at step 7659: 14.590621948242188
Loss at step 7679: 12.20016098022461
Loss at step 7699: 0.0
Loss at step 7719: 0.0
Loss at step 7739: 0.0
Average loss at epoch 17: 8.916982574495918
Took: 13.504043102264404 seconds
Accuracy at epoch 17: 0.9785 
Took: 0.7479231357574463 seconds
Loss at step 7759: 23.192916870117188
Loss at step 7779: 18.947620391845703
Loss at step 7799: 0.0
Loss at step 7819: 14.19647216796875
Loss at step 7839: 16.239910125732422
Loss a

Loss at step 11659: 0.0
Loss at step 11679: 0.0
Loss at step 11699: 0.0
Loss at step 11719: 14.354545593261719
Loss at step 11739: 0.0
Loss at step 11759: 0.0
Loss at step 11779: 0.0
Loss at step 11799: 6.890388488769531
Loss at step 11819: 0.0
Loss at step 11839: 16.228740692138672
Loss at step 11859: 0.0
Loss at step 11879: 0.0
Loss at step 11899: 0.0
Loss at step 11919: 0.0
Loss at step 11939: 7.5826873779296875
Loss at step 11959: 0.0
Loss at step 11979: 0.0
Loss at step 11999: 0.0
Loss at step 12019: 0.0
Loss at step 12039: 0.0
Average loss at epoch 27: 4.242783787411226
Took: 13.512002944946289 seconds
Accuracy at epoch 27: 0.9822 
Took: 0.7501888275146484 seconds
Loss at step 12059: 0.0
Loss at step 12079: 15.846878051757812
Loss at step 12099: 0.0
Loss at step 12119: 0.0
Loss at step 12139: 0.0
Loss at step 12159: 0.0
Loss at step 12179: 0.0
Loss at step 12199: 0.0
Loss at step 12219: 5.127838134765625
Loss at step 12239: 0.0
Loss at step 12259: 8.277763366699219
Loss at step 1